In [ ]:
import os
import pandas as pd
import numpy as np
import quiche as qu
import scanpy as sc
import anndata
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline
from supplementary_plot_helpers import *

sc.set_figure_params(dpi = 400, dpi_save = 400, fontsize = 14)

In [ ]:
save_directory = os.path.join('publications', 'supplementary_figures', 'supplementary_figure16')
qu.pp.make_directory(save_directory)

In [ ]:
sns.set_style('ticks')
adata_spain = anndata.read_h5ad(os.path.join('data', 'Zenodo', 'spain_preprocessed.h5ad'))
adata_spain.obs['Relapse'] = adata_spain.obs['Relapse'].astype('int').astype('str')
cell_table_clusters = pd.read_csv('/Volumes/Shared/Noah Greenwald/TNBC_Cohorts/SPAIN/intermediate_files/mask_dir/individual_masks-no_tagg_tls/cell_annotation_mask.csv')

merged_df = pd.merge(adata_spain.obs[['fov', 'label']], cell_table_clusters, on =['fov', 'label'])
adata_spain.obs['compartment'] = merged_df['mask_name'].values
compartment_dict = dict(zip(adata_spain.obs[['Patient_ID', 'compartment']].drop_duplicates()['Patient_ID'], adata_spain.obs[['Patient_ID', 'compartment']].drop_duplicates()['compartment']))

norm_counts_compartment, p_values_df = compute_abundance(adata_spain, 'Patient_ID', 'cell_cluster', 'compartment', compartment_dict, id1 = 'cancer_border', id2 = 'stroma_core')
plot_cell_type_abundance_grid(norm_counts_compartment, p_values_df, 'compartment', 'cell_cluster', fdr_column='FDR_p_value', save_directory=save_directory,order = ['stroma_core', 'cancer_border'], filename_save = 'supplementary_figure16a.pdf')

In [ ]:
adata_spain = anndata.read_h5ad(os.path.join('data', 'Zenodo', 'spain_preprocessed.h5ad'))
adata_spain.obs['Relapse'] = adata_spain.obs['Relapse'].astype('int').astype('str')
align_dir = '/Volumes/Shared/Noah Greenwald/TNBC_Cohorts/SPAIN/intermediate_files/mask_dir/alignment'

cell_table_clusters = pd.read_csv(os.path.join(align_dir, 'ecm_cell_annotations.csv'))
merged_df = pd.merge(adata_spain.obs[['fov', 'label']], cell_table_clusters, on =['fov', 'label'])
adata_spain.obs['compartment'] = merged_df['compartment'].values

compartment_dict = dict(zip(adata_spain.obs[['fov', 'compartment']].drop_duplicates()['fov'], adata_spain.obs[['fov', 'compartment']].drop_duplicates()['compartment']))

norm_counts_compartment, p_values_df = compute_abundance(adata_spain, 'fov', 'cell_cluster', 'compartment', compartment_dict, id1 = 'aligned', id2 = 'unaligned')
plot_cell_type_abundance_grid(norm_counts_compartment, p_values_df, 'compartment', 'cell_cluster', fdr_column='FDR_p_value', save_directory=save_directory,order = ['unaligned', 'aligned'], filename_save = 'supplementary_figure16b.pdf')